# Introduction to NLP Fundatmentals in TensorFlow

NLP has the goal of deriving information out of natural language

Another common term for NLP problem is seq2seq

## Check for GPU

In [1]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


## Downloading helper function inside the folder


! wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

In [5]:
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

## Get a text dataset

The dataset we're going ti be using is Kaggle's introduction to NLP dataset. A classification problem

[Competition Link](https://www.kaggle.com/competitions/nlp-getting-started/overview)

In [6]:
!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

unzip_data('nlp_getting_started.zip')

--2024-03-11 15:38:15--  https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 2404:6800:4009:81f::201b, 2404:6800:4009:820::201b, 2404:6800:4009:824::201b, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2404:6800:4009:81f::201b|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607343 (593K) [application/zip]
Saving to: 'nlp_getting_started.zip'

nlp_getting_started 100%[===================>] 593.11K  --.-KB/s    in 0.1s    

2024-03-11 15:38:16 (4.91 MB/s) - 'nlp_getting_started.zip' saved [607343/607343]



## Become one with the data

In [7]:
# Read thed data
import pandas as pd

train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [8]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac= 1, random_state= 42)

train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
# How many examples of each class
train_df['target'].value_counts()

target
0    4342
1    3271
Name: count, dtype: int64

In [11]:
# How may samples
len(train_df), len(test_df)

(7613, 3263)

In [12]:
# Let's visualise some random training examples
import random
random_index = random.randint(0, len(train_df) - 5)

for row in train_df_shuffled[['text', 'target']][random_index: random_index + 5].itertuples():
    _, text, target = row
    print(f"Target: {'Disaster' if target == 1 else 'Not a disaster'}")
    print(f"Text: {text}")
    print("----\n")


Target: Disaster
Text: The Murderous Story Of AmericaÛªs First Hijacking: Earnest PletchÛªs cold-blooded killing ofÛ_ http://t.co/B9JAxx0vCf
----

Target: Disaster
Text: Hm MT @Ebolatrends: Alabama Home Quarantined Over Possible Ebola Case http://t.co/ihVMtmZXne http://t.co/jLieMrSnnj
----

Target: Not a disaster
Text: This nigga Cyhi diss was what Meek was suppose to do. This shit actually fire
----

Target: Not a disaster
Text: has NOT sunk in that i leave for school in a month
----

Target: Disaster
Text: Virgin galactic crash: early unlocking of brakes triggered structural failure - Irish Examiner http://t.co/ocMCvfDZkv
----



### Split data into training and validation sets

In [13]:
from sklearn.model_selection import train_test_split

# Use train_test_split() to split the trianing data into train and validation dataset

train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled['text'].to_numpy(), train_df_shuffled['target'].to_numpy(), test_size= 0.1, random_state= 42)

In [14]:
len(train_sentences), len(train_labels), len(val_sentences), len(val_labels)

(6851, 6851, 762, 762)

In [15]:
# Check the first 10 sentences
train_sentences[:10], train_labels[:10]

(array(['@mogacola @zamtriossu i screamed after hitting tweet',
        'Imagine getting flattened by Kurt Zouma',
        '@Gurmeetramrahim #MSGDoing111WelfareWorks Green S welfare force ke appx 65000 members har time disaster victim ki help ke liye tyar hai....',
        "@shakjn @C7 @Magnums im shaking in fear he's gonna hack the planet",
        'Somehow find you and I collide http://t.co/Ee8RpOahPk',
        '@EvaHanderek @MarleyKnysh great times until the bus driver held us hostage in the mall parking lot lmfao',
        'destroy the free fandom honestly',
        'Weapons stolen from National Guard Armory in New Albany still missing #Gunsense http://t.co/lKNU8902JE',
        '@wfaaweather Pete when will the heat wave pass? Is it really going to be mid month? Frisco Boy Scouts have a canoe trip in Okla.',
        'Patient-reported outcomes in long-term survivors of metastatic colorectal cancer - British Journal of Surgery http://t.co/5Yl4DC1Tqt'],
       dtype=object),
 array([0,

## Converting text into numbers